In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
path = "../data/partial_files/"
elements_list = ["players_info", "match_info", "players_lanes", "player_laning_stats",
                 "player_flair_stats", "champion_bans", "champion_picks",
                 "player_combat_stats", "player_objective_stats", "players_champions"]
files_list = os.listdir(path)

# Concatenate the dataframes found for each element

In [3]:
frames = {}
for element in elements_list:
    to_concat = []
    for file in files_list:
        if element in file:
            to_concat.append(pd.read_pickle(path+file))
    frames[element] = pd.concat(to_concat, ignore_index=True)

# Check the data for duplicates or weird values

In [4]:
frames.keys()

dict_keys(['players_info', 'match_info', 'players_lanes', 'player_laning_stats', 'player_flair_stats', 'champion_bans', 'champion_picks', 'player_combat_stats', 'player_objective_stats', 'players_champions'])

In [5]:
frames["match_info"].duplicated().sum()

0

In [6]:
frames["players_info"].duplicated().sum()

52600

In [7]:
frames["players_champions"].head()

,account_id,champion,won
0,5HoOasT7gDFHs3Gr431G2Bd-EN9TfKXrwW4HE11ilp2z4w,131,1
1,ErnjL1gpip46VZJm3oI_t4V5nd7rcwOSm5Ti_gHAkNshj6...,64,1
2,jMALkM1WqOXfJSeofHlzNPleWi_gqq3_X4_lp-BriQxDiCk,24,1
3,VlmeMjfKi2sWiQVDyQ9rAeAZf90WT5OaEmUThGZUWU6EcQ,432,1
4,w2R2-UMM6cwJy3F-fpiWDIq-Lat9bT6q_-J6HpZlOrIxTt8,236,1


In [8]:
frames["players_lanes"].head()

,account_id,lane,won
0,7UTpYZvoj06Si113SIlBe-jyteHrh-XRaYzuYfXWentoKm...,JUNGLE,0
1,fG8JDk5zVxKmoAMUqBIE8nbgMqzn8zuJrDJFPslxAg,BOTTOM,0
2,Kc97-m0MqgpSk3DFoY17uq39_Roh9Qvi-xtoEFXPsMhEWPY,SUPPORT,0
3,WmwA8a6PWVm1SkA3JWpID3CDFAbqxjrsU9f345u3_qR12c...,MIDDLE,0
4,4NI6_UJFRWXe6swXbvsN9dQl6ORPjOfU1EA7ybGJjuc,TOP,0


In [9]:
frames["player_laning_stats"].duplicated().sum()

0

In [10]:
frames["player_combat_stats"].duplicated().sum()

0

In [11]:
frames["player_objective_stats"].duplicated().sum()

0

In [12]:
frames["player_flair_stats"].duplicated().sum()

0

In [13]:
frames["champion_bans"]["champion"] = frames["champion_bans"]["champion"].replace(-1, "no ban")

In [14]:
frames["champion_picks"].duplicated().sum()

0

# Clean the data from duplicates and check dtypes and store the cleaned data to our data folder

In [15]:
final_path = "../data/"

**players_info**

In [16]:
frames["players_info"].dtypes

account_id     object
summoner_id    object
region         object
name           object
dtype: object

In [17]:
frames["players_info"] = frames["players_info"].drop_duplicates()
frames["players_info"].to_csv(final_path+"players_info.csv", index=False)

**match_info**

In [18]:
frames["match_info"].dtypes
frames["match_info"].to_csv(final_path+"match_info.csv", index=False)

**champion_statistics**

In [19]:
frames["champion_bans"].to_csv(final_path+"champion_bans.csv", index=False)
frames["champion_picks"].to_csv(final_path+"champion_picks.csv", index=False)

**player_statistics**

In [20]:
frames["players_lanes"].to_csv(final_path+"player_lanes.csv", index=False)
frames["players_champions"].to_csv(final_path+"players_champions.csv", index=False)

# in game
# drop duplicates
frames["player_laning_stats"] = frames["player_laning_stats"].drop_duplicates()
frames["player_combat_stats"] = frames["player_combat_stats"].drop_duplicates()
frames["player_objective_stats"] = frames["player_objective_stats"].drop_duplicates()
frames["player_flair_stats"] = frames["player_flair_stats"].drop_duplicates()


# store files
frames["player_laning_stats"].to_csv(final_path+"player_laning_stats.csv", index=False)
frames["player_combat_stats"].to_csv(final_path+"player_combat_stats.csv", index=False)
frames["player_objective_stats"].to_csv(final_path+"player_objective_stats.csv", index=False)
frames["player_flair_stats"].to_csv(final_path+"player_flair_stats.csv", index=False)

**Merge the data**

In [23]:
shared = ["match_id", "account_id", "region", "champion", "lane", "won"]

complete_df = (pd.merge(frames["player_laning_stats"], frames["player_combat_stats"], on=shared, how="left")
                .merge(frames["player_objective_stats"], on=shared, how="left")
                .merge(frames["player_flair_stats"], on=shared, how="left")
                .fillna(0))
complete_df.head()

,match_id,account_id,region,champion,lane,xppm_10,cspm_10,goldpm_10,dmg_takenpm_10,won,...,jungle_cs,jungle_invaded,wards_placed,wards_killed,killing_sprees,longest_time_alive,double_kills,triple_kills,quadra_kills,penta_kills
0,4671787510,7UTpYZvoj06Si113SIlBe-jyteHrh-XRaYzuYfXWentoKm...,EUW1,62,JUNGLE,267.3,0.5,261.4,654.3,0,...,57.0,4.0,6.0,6.0,1,495,0,0,0,0
1,4671787510,fG8JDk5zVxKmoAMUqBIE8nbgMqzn8zuJrDJFPslxAg,EUW1,222,BOTTOM,342.6,6.9,345.3,243.3,0,...,13.0,0.0,6.0,2.0,2,475,0,0,0,0
2,4671787510,Kc97-m0MqgpSk3DFoY17uq39_Roh9Qvi-xtoEFXPsMhEWPY,EUW1,412,SUPPORT,274.1,1.1,289.8,311.0,0,...,0.0,0.0,31.0,2.0,1,807,0,0,0,0
3,4671787510,WmwA8a6PWVm1SkA3JWpID3CDFAbqxjrsU9f345u3_qR12c...,EUW1,555,MIDDLE,359.4,4.3,341.8,618.7,0,...,0.0,0.0,4.0,3.0,2,358,0,0,0,0
4,4671787510,4NI6_UJFRWXe6swXbvsN9dQl6ORPjOfU1EA7ybGJjuc,EUW1,54,TOP,384.4,5.6,207.2,335.3,0,...,0.0,0.0,9.0,2.0,1,726,0,0,0,0


In [24]:
complete_df.to_pickle("../data/merged_stats.pkl", protocol=4)